# Запросы на отрезках

## Префиксные последовательности

**Определение:** Префиксная последовательность от последовательности $A_n$ и обратимой операции $\circ$ - это

\begin{align}
$S_n(\omega) =$
\begin{cases}
0, & \mbox{ } n = 0 \\
A_0, & \mbox{ } n = 1 \\
S_{n - 1} \circ A_{n - 1}, & \mbox{ } n > 1
\end{cases}
\end{align}

Важнейшим свойством префиксной последовательности является то, что $A_l \circ A_{l + 1} \circ ... \circ A_r = S_r \circ S_l^{-1}$. Это позволяет выполнять запросы на отрезке массива за $O(1)$

__Замечание:__ Свойством обратимости обладают например сумма, произведение, исключающее или, сложение по модулю и другие групповые операции. Зато им не обладают min/max

Построение префиксной последовательности:

In [14]:
from collections.abc import Callable

def build(A: list, f: Callable[[int, int], int]) -> list:
    n = len(A)
    S = [0] * (n + 1)
    S[1] = A[0]

    for i in range(2, n + 1):
        S[i] = f(S[i - 1], A[i - 1])

    return S

A = [1, 4, 12, 13, 1, 56, 17]
f = lambda x, y: x + y
f_reversed = lambda x, y: y - x
S = build(A, f)
print(S)

[0, 1, 5, 17, 30, 31, 87, 104]


Добавление элементов в префиксную последовательность

In [13]:
def append(S: list, a, f: Callable[[int, int], int]):
    S.append(f(S[-1], a))

append(S, 11, f)
print(S)

[0, 1, 5, 17, 30, 31, 87, 104, 115, 126]


Запрос суммы на отрезке от l до r

In [16]:
l = 2
r = 5
print(f_reversed(S[l], S[r]))

26


## Дерево отрезков

**Определение** Дерево отрезков от последовательности $A_n$ и ассоциативной функции $\phi$ - это дерево, листьями которого являются элементы исходного массива, а вершинами - пары из объединения интервалов их детей и значений ассоциативной функции $\phi$. У каждой вершины по два ребенка.

In [47]:
n = 7
A = [1, 4, 12, 13, 1, 56, 17]
D = [0] * 3 * n
f = lambda x, y: x + y

Добавление элемента в дерево

In [50]:
def update(v: int, lv: int, rv: int, i: int, val: int):
    if i < lv or i >= rv:
        return
    if lv + 1 == rv:
        D[v] = val
        return
    mid = (lv + rv) // 2
    update(v * 2 + 1, lv, mid, i, val)
    update(v * 2 + 2, mid, rv, i, val)
    D[v] = f(D[v * 2 + 1], D[v * 2 + 2])

def build():
    for i in range(n):
        update(0, 0, n, i, A[i])

print(D)

[104, 17, 87, 1, 16, 14, 73, 0, 0, 4, 12, 13, 1, 56, 17, 0, 0, 0, 0, 0, 0]


In [52]:
def query(v: int, lv: int, rv: int, l: int, r: int):
    if l <= lv and rv <= r:
        return D[v]
    if r <= lv or l >= rv:
        return 0
    mid = (lv + rv) // 2
    return f(query(v * 2 + 1, lv, mid, l, r), query(v * 2 + 2, mid, rv, l, r))

query(0, 0, n, 1, 4)

29

## Дерево Фенвика